## Top tag : 0.9 , 120 250 
## B tag : particle net tight 0.6734
## HLT : HLT_IsoMu30


## BKG

In [9]:
#!/usr/bin/env python3
import os
import re
import glob
import csv

# 1) 로그 파일이 들어있는 폴더
LOG_DIR = "/data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/bkg/condorfile/out"

# 2) 결과를 담을 리스트
rows = []

# 3) 각 로그 파일 순회
for logpath in glob.glob(os.path.join(LOG_DIR, "*.log")):
    name = os.path.splitext(os.path.basename(logpath))[0]  # 파일명 베이스
    with open(logpath, 'r') as f:
        text = f.read()

    # 4) 정규표현식으로 숫자 추출
    m_pass = re.search(r"Passed events\s*:\s*([0-9]+)", text)
    m_weight = re.search(r"Per-event weight\s*:\s*([0-9.eE\+\-]+)", text)
    m_exp = re.search(r"Expected yield\s*:\s*([0-9.eE\+\-]+)", text)

    # 5) 모두 찾았을 때만 추가
    if m_pass and m_weight and m_exp:
        passed      = int(m_pass.group(1))
        weight      = float(m_weight.group(1))
        expected    = float(m_exp.group(1))
        rows.append({
            "sample": name,
            "Passed events": passed,
            "Per-event weight": weight,
            "Expected yield": expected
        })
    else:
        print(f"Skipping {name}: incomplete data")

# 6) Expected yield 기준으로 내림차순 정렬
rows.sort(key=lambda x: x["Expected yield"], reverse=True)

# → 여기서 각 컬럼 합계 계산
total_passed   = sum(r["Passed events"]      for r in rows)
total_weight   = sum(r["Per-event weight"]   for r in rows)
total_expected = sum(r["Expected yield"]      for r in rows)

# → 합계 행을 추가
rows.append({
    "sample":          "Total",
    "Passed events":   total_passed,
    "Per-event weight": total_weight,
    "Expected yield":  total_expected
})

# 7) CSV로 저장 (Total 행 포함)
out_csv = "summary.csv"
with open(out_csv, 'w', newline='') as csvfile:
    fieldnames = ["sample", "Passed events", "Per-event weight", "Expected yield"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)

print(f"Wrote {len(rows)} entries (including total row) to {out_csv}")

Skipping SingleMuon_C: incomplete data
Skipping SingleMuon_B: incomplete data
Skipping JetMET_C: incomplete data
Skipping EGamma_D: incomplete data
Skipping JetMET_D: incomplete data
Skipping EGamma_B: incomplete data
Skipping Tau_B: incomplete data
Skipping Jpsito2Mu: incomplete data
Skipping Tau_C: incomplete data
Skipping Muon_C: incomplete data
Skipping Tau_D: incomplete data
Skipping EGamma_C: incomplete data
Skipping Muon_D: incomplete data
Skipping JetMET_B: incomplete data
Wrote 46 entries (including total row) to summary.csv


## SIG

In [10]:
#!/usr/bin/env python3
import os
import re
import glob
import csv
import math

# 0) background summary.csv 에서 B 값 읽어오기
BG_SUMMARY = "summary.csv"  # 필요에 따라 경로를 조정하세요
with open(BG_SUMMARY, newline='') as f_bg:
    reader_bg = csv.DictReader(f_bg)
    bg_rows = list(reader_bg)
    # 마지막 행의 "Expected yield" 를 B 로 설정
    B = float(bg_rows[-1]["Expected yield"])

# 1) 로그 파일이 들어있는 폴더
LOG_DIR = "/data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/sig/condorfile/out"

# 2) 결과를 담을 리스트
rows = []

# 3) 각 로그 파일 순회
for logpath in glob.glob(os.path.join(LOG_DIR, "*.log")):
    name = os.path.splitext(os.path.basename(logpath))[0]  # 파일명 베이스
    with open(logpath, 'r') as f:
        text = f.read()

    # 4) 정규표현식으로 숫자 추출
    m_pass   = re.search(r"Passed events\s*:\s*([0-9]+)", text)
    m_weight = re.search(r"Per-event weight\s*:\s*([0-9.eE\+\-]+)", text)
    m_exp    = re.search(r"Expected yield\s*:\s*([0-9.eE\+\-]+)", text)

    # 5) 모두 찾았을 때만 추가
    if m_pass and m_weight and m_exp:
        passed   = int(m_pass.group(1))
        weight   = float(m_weight.group(1))
        expected = float(m_exp.group(1))
        # S_over_sqrtB 계산
        s_over_sqrtB = expected / math.sqrt(B)
        rows.append({
            "sample":          name,
            "Passed events":   passed,
            "Per-event weight": weight,
            "Expected yield":  expected,
            "S_over_sqrtB":    s_over_sqrtB
        })
    else:
        print(f"Skipping {name}: incomplete data")

# 6) Expected yield 기준으로 내림차순 정렬 (필요 없으면 제거)
rows.sort(key=lambda x: x["Expected yield"], reverse=True)

# 7) CSV로 저장 (새 컬럼 포함)
out_csv = "sig_summary.csv"
with open(out_csv, 'w', newline='') as csvfile:
    fieldnames = [
        "sample",
        "Passed events",
        "Per-event weight",
        "Expected yield",
        "S_over_sqrtB"      # 새로 추가된 컬럼
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)

print(f"Wrote {len(rows)} entries to {out_csv}")


Wrote 8 entries to sig_summary.csv


[wrmass] No files found under pattern: /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/bkg/condorfile/result_*/wrmass_counts.csv
[mll] No files found under pattern: /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/bkg/condorfile/result_*/mll_counts.csv
[ptmu1] No files found under pattern: /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/bkg/condorfile/result_*/ptmu1_counts.csv
[ptmu2] No files found under pattern: /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/bkg/condorfile/result_*/ptmu2_counts.csv
[ptt] No files found under pattern: /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUC